In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)
print(y_pred)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 97us/sample - loss: 0.7452 - val_loss: 0.5588
Epoch 2/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.4770 - val_loss: 0.4773
Epoch 3/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4416 - val_loss: 0.4368
Epoch 4/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4230 - val_loss: 0.4273
Epoch 5/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4111 - val_loss: 0.3825
Epoch 6/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.3989 - val_loss: 0.3725
Epoch 7/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.3963 - val_loss: 0.3741
Epoch 8/20
11610/11610 [==============================] - 0s 27us/sample - loss: 0.3875 - val_loss: 0.3696
Epoch 9/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.3923 - val_l

In [4]:
# Wide and deep neural net.
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model2 = keras.Model(inputs=[input_], outputs=[output])
model2.compile(loss="mean_squared_error", optimizer="Adam")
print(model2.summary())
history2 = model2.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test2 = model2.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model2.predict(X_new)
print(y_pred)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 30)           270         input_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 30)           930         dense_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 38)           0           input_1[0][0]                    
                                                                 dense_3[0][0]                

In [5]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model3 = keras.Model(inputs=[input_A, input_B], outputs=[output])
model3.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]
history = model3.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model3.evaluate((X_test_A, X_test_B), y_test)
y_pred = model3.predict((X_new_A, X_new_B))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 0s 37us/sample - loss: 2.0348 - val_loss: 1.0947
Epoch 2/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.7172 - val_loss: 0.6339
Epoch 3/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.6147 - val_loss: 0.5724
Epoch 4/20
11610/11610 [==============================] - 0s 31us/sample - loss: 0.5818 - val_loss: 0.5471
Epoch 5/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5619 - val_loss: 0.5308
Epoch 6/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5465 - val_loss: 0.5174
Epoch 7/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5341 - val_loss: 0.5055
Epoch 8/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5233 - val_loss: 0.4955
Epoch 9/20
11610/11610 [==============================] - 0s 28us/sample - loss: 0.5138 - val_l

In [6]:
model3.layers

In [7]:
model3.layers[1]

In [8]:
hidden1 = model3.layers[1]

In [9]:
hidden1.name

'dense_5'

In [10]:
model3.get_layer('dense_5') is hidden1

True

In [13]:
import os
if os.path.exists("model4.hdf5"):
    model4 = keras.models.load_model("model4.hdf5")
else:
    main_output = keras.layers.Dense(1, name="main_output")(concat)
    aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
    model4 = keras.Model(inputs=[input_A, input_B], outputs=[main_output, aux_output])
    model4.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))

    history = model4.fit((X_train_A, X_train_B), [y_train, y_train], epochs=20,
                        validation_data=((X_valid_A, X_valid_B), [y_valid, y_valid]))
total_loss, main_loss, aux_loss = model4.evaluate([X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model4.predict([X_new_A, X_new_B])
print(total_loss, main_loss, aux_loss)
print(y_pred_main, y_pred_aux)
if not os.path.exists("model4.hdf5"):
    model4.save("model4.hdf5")

5160/5160 [==============================] - 0s 23us/sample - loss: 0.4981 - main_output_loss: 0.4637 - aux_output_loss: 0.7973
0.4981348545514336 0.46374595 0.7973324
[[1.9625449]
 [1.5608535]
 [3.0432026]] [[2.3038797]
 [1.7771248]
 [2.6100736]]


In [15]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model5 = WideAndDeepModel()
model5.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))
main_output, aux_output = model5(inputs=[input_A, input_B])
history = model5.fit((X_train_A, X_train_B), [y_train, y_train], epochs=20,
                     validation_data=((X_valid_A, X_valid_B), [y_valid, y_valid]))
total_loss, main_loss, aux_loss = model5.evaluate([X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model5.predict([X_new_A, X_new_B])
print(total_loss, main_loss, aux_loss)
print(y_pred_main, y_pred_aux)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 47us/sample - loss: 2.6605 - output_1_loss: 2.3883 - output_2_loss: 5.1018 - val_loss: 3.1175 - val_output_1_loss: 2.8633 - val_output_2_loss: 5.3986
Epoch 2/20
11610/11610 [==============================] - 0s 34us/sample - loss: 1.0803 - output_1_loss: 0.8150 - output_2_loss: 3.4674 - val_loss: 1.2949 - val_output_1_loss: 0.9618 - val_output_2_loss: 4.2915
Epoch 3/20
11610/11610 [==============================] - 0s 38us/sample - loss: 0.8427 - output_1_loss: 0.6745 - output_2_loss: 2.3572 - val_loss: 0.9592 - val_output_1_loss: 0.6801 - val_output_2_loss: 3.4682
Epoch 4/20
11610/11610 [==============================] - 0s 31us/sample - loss: 0.7407 - output_1_loss: 0.6212 - output_2_loss: 1.8142 - val_loss: 0.8397 - val_output_1_loss: 0.6151 - val_output_2_loss: 2.8600
Epoch 5/20
11610/11610 [==============================] - 0s 35us/sample - loss: 0.6876 - output_1_loss: 0.